In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import yfinance as yf
import fredapi as fa
from datetime import date, timedelta
from Sector_Exposure import Rate_Exposures
import numpy as np
import matplotlib.pyplot as plt
import Useful_Functions as u
from statsmodels.tsa.stattools import adfuller

# sectors_and_rates
cutoff = .7

#sector_df = pd.read_csv("2006.csv", index_col="Date")

sector_df = pd.read_csv("sectors_and_rates.csv", index_col="Date")

train_df = sector_df[:int(len(sector_df)*cutoff)]
test_df = sector_df[int(len(sector_df)*cutoff):]

exposure = Rate_Exposures(train_df, 252, 30, ['Real Yield', 'Yield Curve'], "SP-500")
for_scores = Rate_Exposures(sector_df, 252, 30, ['Real Yield', 'Yield Curve'], "SP-500")

ry_scores_test = for_scores.scores['Real Yield'][int(len(sector_df)*cutoff):]
yc_scores_test = for_scores.scores['Yield Curve'][int(len(sector_df)*cutoff):]
train_results_df = exposure.mean_returns.copy()


In [7]:

buys = []
exp_ret = []

top_n_sectors = 3

for num, (ry, yc) in enumerate(zip(ry_scores_test, ry_scores_test)):
    
    #if num + 252 < len(ry_scores_test):
    
    
    if abs(ry) <= 6 and abs(yc) <= 5:
        
        try:
            #buys.append(train_results_df.loc[(ry,yc)].loc[train_results_df.loc[(ry,yc)] > 0.0].index.to_list())
            ok = set(train_results_df.loc[(ry,yc)].sort_values().index[-top_n_sectors:].to_list())
            buys.append(ok)
            exp_ret.append(train_results_df.loc[(ry,yc)].sort_values()[-top_n_sectors:].mean())
            
        except:
            try:
                #buys.append(train_results_df.loc[(ry,yc-1)].loc[train_results_df.loc[(ry,yc-1)] > 0.0].index.to_list())
                ok = set(train_results_df.loc[(ry,yc-1)].sort_values().index[-top_n_sectors:].to_list())
                buys.append(ok)
                exp_ret.append(train_results_df.loc[(ry,yc-1)].sort_values()[-top_n_sectors:].mean())
            except:
                #buys.append(train_results_df.loc[(ry,yc+1)].loc[train_results_df.loc[(ry,yc+1)] > 0.0].index.to_list())
                ok = train_results_df.loc[(ry,yc+1)].sort_values().index[-top_n_sectors:].to_list()
                buys.append(ok)
                exp_ret.append(train_results_df.loc[(ry,yc+1)].sort_values()[-top_n_sectors:].mean())
    
    elif abs(ry) > 6 and abs(yc) <= 5:
        
        if ry > 6:
            #buys.append(train_results_df.loc[(6,yc)].loc[train_results_df.loc[(6,yc)] > 0.0].index.to_list())
            ok = set(train_results_df.loc[(6,yc)].sort_values().index[-top_n_sectors:].to_list())
            buys.append(ok)
            exp_ret.append(train_results_df.loc[(6,yc)].sort_values()[-top_n_sectors:].mean())
        if ry < -6:
            #buys.append(train_results_df.loc[(-6,yc)].loc[train_results_df.loc[(-6,yc)] > 0.0].index.to_list())
            ok = set(train_results_df.loc[(-6,yc)].sort_values().index[-top_n_sectors:].to_list())
            buys.append(ok)
            exp_ret.append(train_results_df.loc[(-6,yc)].sort_values()[-top_n_sectors:].mean())
    
    elif abs(ry) <= 6 and abs(yc) > 5:
        
        #print((ry, yc))
        
        if yc > 5:
            #buys.append(train_results_df.loc[(ry,5)].loc[train_results_df.loc[(ry,5)] > 0.0].index.to_list())
            ok = set(train_results_df.loc[(ry,5)].sort_values().index[-top_n_sectors:].to_list())            
            buys.append(ok)
            exp_ret.append(train_results_df.loc[(ry,5)].sort_values()[-top_n_sectors:].mean())
        elif yc < -5:
            #buys.append(train_results_df.loc[(ry,-5)].loc[train_results_df.loc[(ry,-5)] > 0.0].index.to_list())
            ok = set(train_results_df.loc[(ry,-5)].sort_values().index[-top_n_sectors:].to_list())
            buys.append(ok)
            exp_ret.append(train_results_df.loc[(ry,-5)].sort_values()[-top_n_sectors:].mean())
    
    elif abs(ry) > 6 and abs(yc) > 5:
        
        if ry > 6 and yc > 5:
            #buys.append(train_results_df.loc[(6,5)].loc[train_results_df.loc[(6,5)] > 0.0].index.to_list())
            ok = set(train_results_df.loc[(6,5)].sort_values().index[-top_n_sectors:].to_list())
            buys.append(ok)
            exp_ret.append(train_results_df.loc[(6,5)].sort_values()[-top_n_sectors:].mean())
        elif ry < -6 and yc < -5:
            #buys.append(train_results_df.loc[(-6,-5)].loc[train_results_df.loc[(-6,-5)] > 0.0].index.to_list())
            ok = set(train_results_df.loc[(-6,-5)].sort_values().index[-top_n_sectors:].to_list())
            buys.append(ok)
            exp_ret.append(train_results_df.loc[(-6,-5)].sort_values()[-top_n_sectors:].mean())
        elif ry > 6 and yc < -5:
            #buys.append(train_results_df.loc[(6,-5)].loc[train_results_df.loc[(6,-5)] > 0.0].index.to_list())
            ok = set(train_results_df.loc[(6,-5)].sort_values().index[-top_n_sectors:].to_list())
            buys.append(ok)
            exp_ret.append(train_results_df.loc[(6,-5)].sort_values()[-top_n_sectors:].mean())
        elif ry < -6 and yc > 5:
            #buys.append(train_results_df.loc[(-6,5)].loc[train_results_df.loc[(-6,5)] > 0.0].index.to_list())
            ok = set(train_results_df.loc[(-6,5)].sort_values().index[-top_n_sectors:].to_list())
            buys.append(ok)
            exp_ret.append(train_results_df.loc[(-6,5)].sort_values()[-top_n_sectors:].mean())


expected_results = pd.DataFrame([buys, exp_ret]).T
expected_results.columns = ['exp_secs', 'exp_rets']
expected_results.index = ry_scores_test.index
#expected_results = expected_results[:-120]

expected_results

,exp_secs,exp_rets
2019-12-04,"{Semiconductors, Financials, Information Techn...",0.017518
2019-12-05,"{Semiconductors, Financials, Information Techn...",0.01314
2019-12-06,"{Semiconductors, Financials, Information Techn...",0.01314
2019-12-09,"{Semiconductors, Financials, Information Techn...",0.01314
2019-12-10,"{Semiconductors, Financials, Information Techn...",0.01314
...,...,...
2022-05-31,"{Aerospace, Semiconductors, Health Care}",0.014191
2022-06-01,"{Aerospace, Semiconductors, Health Care}",0.014191
2022-06-02,"{Aerospace, Semiconductors, Health Care}",0.014191
2022-06-03,"{Aerospace, Semiconductors, Health Care}",0.014191


In [8]:

actual_results_exposure = Rate_Exposures(test_df, 252, 30, ['Real Yield', 'Yield Curve'], "SP-500")

actual_returns = actual_results_exposure.forward_returns_daily.iloc[:,:-2]

#  & (actual_returns.index <= "2021-12-13")
actual_returns = actual_returns.loc[ (actual_returns.index >= "2019-11-19")]

actual_returns

,Materials,Industrials,Consumer Discretionary,Consumer Staples,Health Care,Financials,Information Technology,Telecommunication Services,Utilities,Real Estate,Energy,Semiconductors,Aerospace
2019-11-19,-0.040419,-0.022257,0.010654,-0.021745,0.007937,-0.004835,0.018316,-0.006637,-0.015969,-0.039588,0.034716,0.032730,-0.047803
2019-11-20,-0.040626,-0.022498,0.009873,-0.028789,0.007396,-0.007832,0.021499,0.006670,-0.027408,-0.046631,0.024385,0.026618,-0.055119
2019-11-21,-0.040233,-0.020865,0.015346,-0.029496,0.003839,-0.012478,0.026790,0.004933,-0.024373,-0.043399,0.006316,0.059366,-0.040373
2019-11-22,-0.039662,-0.026896,0.007291,-0.026328,0.003534,-0.017545,0.036327,0.007910,-0.027499,-0.036546,-0.011185,0.056678,-0.051912
2019-11-25,-0.041701,-0.028634,0.004726,-0.019772,-0.001765,-0.015460,0.034052,0.010255,-0.017501,-0.035581,-0.004159,0.038535,-0.045662
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-18,0.022238,0.006926,-0.083587,0.003327,0.021664,0.012642,0.008682,-0.038906,0.036034,-0.032277,0.125894,0.066307,-0.025966
2022-04-19,0.025364,0.006127,-0.093059,-0.000781,0.021024,0.006291,0.010234,-0.041516,0.050324,-0.039126,0.177057,0.050876,-0.013111
2022-04-20,0.026545,0.001029,-0.073881,-0.019117,-0.001735,-0.004542,0.016647,0.001769,0.030944,-0.059963,0.151755,0.069382,-0.012124
2022-04-21,0.034871,0.008617,-0.086504,-0.026201,-0.000527,-0.003005,0.010900,0.003728,0.043815,-0.063843,0.203835,0.067561,0.017022


In [9]:

actual_secs = []
actual_ret = []
predicted_sector_performance = []
correct_sector_performance = []

for d in actual_returns.index:
    
    # .sort_values().index[-1]
    #actual_results.append(actual_returns.loc[d].loc[actual_returns.loc[d] > 0.0].index.to_list())
    ok = set(actual_returns.loc[d].sort_values().index[-top_n_sectors:].to_list())
    actual_secs.append(ok)
    
    actual_ret.append(actual_returns.loc[d].sort_values()[-top_n_sectors:].mean())
    
    #if d in expected_results.index:
    predicted_sector_performance.append(actual_returns.loc[d][list(expected_results['exp_secs'].loc[d])].mean())

actual_secs = pd.Series(actual_secs, index = actual_returns.index)
actual_ret = pd.Series(actual_ret, index = actual_returns.index)

actual_results = pd.DataFrame()
actual_results['actual_secs'] = actual_secs
actual_results['actual_rets'] = actual_ret

#actual_results = actual_results.loc[ (actual_results.index >= "2018-11-19") & (actual_results.index <= "2021-12-13")]


KeyboardInterrupt



In [ ]:

correct_sectors = []
mean_squared_error_return = []
return_error_regular = []

for pred, act in zip(expected_results['exp_secs'], actual_results['actual_secs']):
    
    sect = pred.intersection(act)
    correct_sectors.append(len(sect))
    
    
for pred, act in zip(expected_results['exp_rets'], actual_results['actual_rets']):
    
    mean_squared_error_return.append((pred-act)**2)
    return_error_regular.append(pred-act) 
    

correct_sectors = pd.Series(correct_sectors, index = actual_results['actual_secs'].index)
mean_squared_error_return = pd.Series(mean_squared_error_return, index = actual_results['actual_secs'].index)
return_error_regular = pd.Series(return_error_regular, index = actual_results['actual_secs'].index)


In [ ]:

total_performance = pd.DataFrame(index = actual_results['actual_secs'].index)
total_performance['predicted_secs'] = expected_results.exp_secs
total_performance['actual_secs'] = actual_secs
total_performance['sector_prediction_performance'] = correct_sectors
total_performance['predicted_sector_performance'] = predicted_sector_performance
total_performance['correct_sector_performance'] = actual_results['actual_rets']
total_performance['mean_squared_error_return_prediction'] = mean_squared_error_return
total_performance['return_error_regular'] = return_error_regular
total_performance = total_performance.dropna()

monthly_performance = u.get_monthly_data(total_performance)


In [ ]:


(total_performance.predicted_sector_performance > 0.0).value_counts()


344 / (344+173)

#total_performance.predicted_sector_performance.mean() * ((252 - 30)/30)



0.6653771760154739

In [ ]:

#u.barplot_1d(total_performance.predicted_sector_performance)


#plt.scatter(total_performance.sector_prediction_performance, total_performance.predicted_sector_performance)

ok = total_performance[['sector_prediction_performance', 'predicted_sector_performance']]

ok.predicted_sector_performance.loc[ok.sector_prediction_performance == 4].mean()
#ok.predicted_sector_performance.loc[ok.sector_prediction_performance == 0].mean()
#total_performance

0.03611499623263311

In [ ]:


cutoff = .7

factor_df = pd.read_excel("factors_and_scores.xlsx", index_col="Date")

factor_train = factor_df[:int(len(factor_df)*cutoff)]
factor_test = factor_df[int(len(factor_df)*cutoff):]


,DEFAULT,QUALITY,MOMENTUM,BOOK_TO_PRICE,SIZE,CASH_TO_ASSETS,ry_scores,yc_scores
Date,,,,,,,,
2011-02-10,-0.1495,0.0886,-0.0021,0.0025,-0.0523,0.0101,3,2
2011-02-11,-0.1588,0.1091,-0.0002,0.0090,-0.0527,0.0060,2,1
2011-02-14,-0.1776,0.1124,-0.0027,0.0057,-0.0471,0.0004,1,0
2011-02-15,-0.1631,0.1311,-0.0020,0.0102,-0.0536,0.0104,1,1
2011-02-16,-0.1534,0.1258,-0.0035,0.0086,-0.0559,0.0193,2,1
...,...,...,...,...,...,...,...,...
2018-02-05,0.0160,0.0819,0.0132,-0.0999,-0.0354,0.0812,3,0
2018-02-06,0.0191,0.0849,0.0125,-0.1019,-0.0391,0.1002,3,0
2018-02-07,0.0119,0.0932,0.0117,-0.0920,-0.0383,0.0918,4,1


In [239]:
"""

MONTHLY DATA NOW

"""

# sectors_and_rates
cutoff = .7

#sector_df = pd.read_csv("2006.csv", index_col="Date")

sector_df = pd.read_csv("sectors_and_rates.csv", index_col="Date")

#factor_df = pd.read_csv("regular_factors_and_rates.csv", index_col="Date")

train_df = sector_df[:int(len(sector_df)*cutoff)]
test_df = sector_df[int(len(sector_df)*cutoff):]

forward = 20

exposure = Rate_Exposures(train_df, 252, forward, ['Real Yield', 'Yield Curve'], "SP-500")
for_scores = Rate_Exposures(sector_df, 252, forward, ['Real Yield', 'Yield Curve'], "SP-500")

# [int(len(u.get_monthly_data(sector_df))*cutoff):]
ry_scores_test_monthly = for_scores.monthly_scores['Real Yield']
yc_scores_test_monthly = for_scores.monthly_scores['Yield Curve']
ry_scores_test_monthly = ry_scores_test_monthly.loc[ ry_scores_test_monthly.index > train_df.index[-1]]
yc_scores_test_monthly = yc_scores_test_monthly.loc[ yc_scores_test_monthly.index > train_df.index[-1]]
#ry_scores_test_monthly_test = ry_scores_test_monthly.loc[ ry_scores_test_monthly.index <= train_df.index[-1]]
#yc_scores_test_monthly_test = yc_scores_test_monthly.loc[ yc_scores_test_monthly.index <= train_df.index[-1]]


train_results_df = exposure.mean_returns_monthly.copy()
#train_results_df

actual_results_exposure = Rate_Exposures(test_df, 252, forward, ['Real Yield', 'Yield Curve'])
actual_returns = actual_results_exposure.forward_returns_monthly.iloc[:,:-2]
actual_returns = actual_returns.loc[ (actual_returns.index >= ry_scores_test_monthly.index[0]) & (actual_returns.index <= ry_scores_test_monthly.index[-1])]
actual_returns['Real Yield_scores'] = ry_scores_test_monthly
actual_returns['Yield Curve_scores'] = yc_scores_test_monthly
actual_returns['scores'] = [(i,j) for i,j in zip(actual_returns['Real Yield_scores'], actual_returns['Yield Curve_scores'])]
actual_returns = actual_returns.drop(columns = ["Real Yield_scores", 'Yield Curve_scores'])


In [240]:

real_train_results = exposure.forward_returns_monthly.copy()
real_train_results['scores'] = [(i,j) for i,j in zip(real_train_results['Real Yield_scores'], real_train_results['Yield Curve_scores'])]
real_train_results = real_train_results.dropna()
real_train_results = real_train_results.drop(columns = ['Real Yield_scores', 'Yield Curve_scores'])


In [280]:

def closest_combos(combo, in_index = False):

    assert isinstance(combo, tuple)
    
    min_error = 100
    errors = {}

    for rates in train_results_df.index:
        
        ry_error = abs(combo[0] - rates[0])
        yc_error = abs(combo[1] - rates[1])
        error = ry_error + yc_error
         
        errors[rates] = error
        
        if error <= min_error and error > 0:
            min_error = error
    
    mins = list(filter(lambda x: errors[x]==min_error, errors))
    
    if in_index:
        mins.append(combo)
    
    if len(mins) <= 2:
        
        more_mins = list(filter(lambda x: errors[x]==min_error+1, errors))
        
        extended = list( set(mins).union(set(more_mins)) )
        
        if len(extended) <= 3:
            
            even_more_mins = list(filter(lambda x: errors[x]==min_error+2, errors))
            
            return list( set(more_mins).union(set(even_more_mins)) )
        
        return extended
    
    return mins


In [282]:

predicted_best_sector = []
exp_ret = []
scores = []
top_n_sectors = 1

for num, (ry, yc) in enumerate(zip(ry_scores_test_monthly, yc_scores_test_monthly)):
    
    # train_results_df.index
    
    #scores = [(i,j) for i,j in zip(real_train_results['Real Yield_scores'], real_train_results['Yield Curve_scores'])]
    #print(real_train_results['scores'])
    if ((ry, yc) == real_train_results['scores']).any():
        
        combos = closest_combos((ry, yc), True)
    else:
        combos = closest_combos((ry, yc))
    
    #top_secs = train_results_df.loc[combos].mean().sort_values()[-top_n_sectors:]
    
    top_secs = real_train_results.iloc[:,:-1].loc[real_train_results['scores'].isin(combos)].mean().sort_values()[-top_n_sectors:]
    
    predicted_best_sector.append(top_secs.index.to_list())
    exp_ret.append(top_secs.mean())
    scores.append((ry,yc))
    
    update_index = ry_scores_test_monthly.index[num]
    
    if update_index <= pd.Period("2022-04"):
        real_train_results = pd.concat([real_train_results, pd.DataFrame(actual_returns.loc[update_index]).T.drop(columns = ["SP-500"])])
    
    #print(len(real_train_results))
    

big_df = pd.DataFrame(index = ry_scores_test_monthly.index)
big_df['scores'] = scores
big_df['exp_rets_predicted'] = exp_ret
big_df['pred_top_secs'] = predicted_best_sector


In [283]:

actual_secs = []
actual_ret = []
predicted_sector_performance = []
correct_sector_performance = []

for d in actual_returns.index:
    top_actual_secs = actual_returns.iloc[:,:-1].loc[str(d)].sort_values()
    actual_secs.append(top_actual_secs.index[-top_n_sectors:].to_list())
    actual_ret.append(top_actual_secs[-top_n_sectors:].mean())
    predicted_sector_performance.append(actual_returns.iloc[:,:-1].loc[str(d)][list(big_df['pred_top_secs'].loc[str(d)])].mean())

actual_secs = pd.Series(actual_secs, index = actual_returns.index)
actual_ret = pd.Series(actual_ret, index = actual_returns.index)

big_df['actual_top_secs'] = actual_secs
big_df['actual_rets'] = actual_ret

big_df = big_df.dropna()


In [284]:

correct_sectors = []
mean_squared_error_return = []
return_error_regular = []


for pred, act in zip(big_df['pred_top_secs'], big_df['actual_top_secs']):
    sect = set(pred).intersection(set(act))
    correct_sectors.append(len(sect))
    

for pred, act in zip(big_df['exp_rets_predicted'], big_df['actual_rets']):
    
    mean_squared_error_return.append((pred-act)**2)
    return_error_regular.append(pred-act) 
    
big_df['correctly_chosen_sectors'] = correct_sectors
#big_df['correctly_chosen_sectors_above_market'] = correct_sectors_above_market
big_df['predicted_sector_performance'] = predicted_sector_performance
#big_df['correct_sector_performance'] = actual_results['actual_rets']
big_df['mean_squared_error_return_prediction'] = mean_squared_error_return
big_df['return_error_regular'] = return_error_regular
big_df = big_df.dropna()


In [285]:


big_df.correctly_chosen_sectors.mean()



0.3488372093023256

In [286]:


big_df.predicted_sector_performance.loc[ big_df.correctly_chosen_sectors == 1].mean()



0.08151778555904397

In [287]:

big_df.predicted_sector_performance.mean()


0.051806908290418965

In [288]:

top_1 = 7/30
top_2 = 26 / 74
top_3 = 40 / 111

#big_df.correctly_chosen_sectors.plot()

mean_outperform_30_days = big_df.predicted_sector_performance.mean()
mean_yearly_outperformance = mean_outperform_30_days * ((252 - forward)/forward)
#(big_df.predicted_sector_performance > 0.0).value_counts()
mean_yearly_outperformance


0.60096013616886

In [252]:


(big_df.correctly_chosen_sectors > 0).value_counts()



False    23
True     20
Name: correctly_chosen_sectors, dtype: int64

In [266]:



big_df.actual_rets.sum() - big_df.predicted_sector_performance.sum()

big_df



,scores,exp_rets_predicted,pred_top_secs,actual_top_secs,actual_rets,correctly_chosen_sectors,predicted_sector_performance,mean_squared_error_return_prediction,return_error_regular
2018-11,"(2.0, -1.0)",0.016672,[Utilities],[Utilities],0.003626,1,0.003626,0.000170,0.013046
2018-12,"(0.0, -2.0)",0.015759,[Semiconductors],[Consumer Discretionary],0.043507,0,0.036232,0.000770,-0.027748
2019-01,"(-1.0, -1.0)",0.010475,[Semiconductors],[Aerospace],0.117637,0,0.109227,0.011484,-0.107162
2019-02,"(-1.0, -1.0)",0.013467,[Semiconductors],[Information Technology],0.044823,0,0.028513,0.000983,-0.031356
2019-03,"(-3.0, -1.0)",-0.004555,[Semiconductors],[Semiconductors],0.093806,1,0.093806,0.009675,-0.098361
2019-04,"(-2.0, 0.0)",-0.000687,[Consumer Staples],[Consumer Staples],0.012339,1,0.012339,0.000170,-0.013026
2019-05,"(-2.0, -0.0)",-0.000144,[Consumer Staples],[Materials],0.046894,0,0.016098,0.002213,-0.047038
2019-06,"(-3.0, 1.0)",0.033553,[Semiconductors],[Semiconductors],0.096932,1,0.096932,0.004017,-0.063379
2019-07,"(-3.0, -0.0)",0.019618,[Utilities],[Real Estate],0.024847,0,0.008873,0.000027,-0.005229
2019-08,"(-4.0, -4.0)",0.034628,[Semiconductors],[Semiconductors],0.063365,1,0.063365,0.000826,-0.028737


In [279]:



actual_returns.loc['2022-04']

combos = closest_combos((5,-1))


# .mean().sort_values()
# ['Aerospace']

real_train_results.iloc[:,:-1].loc[real_train_results['scores'].isin(combos)].mean().sort_values()
#real_train_results.iloc[:,:-1].loc[real_train_results['scores'].isin(combos)][u.ut]

#combos



Energy                       -0.032147
Materials                    -0.018439
Financials                   -0.014965
Industrials                  -0.010704
Telecommunication Services   -0.010681
Consumer Staples             -0.005966
Aerospace                    -0.005170
Consumer Discretionary       -0.001557
Real Estate                   0.000701
Health Care                   0.001180
Semiconductors                0.001940
Information Technology        0.009577
Utilities                     0.009729
dtype: float64

Index(['1/4/2010', '1/5/2010', '1/6/2010', '1/7/2010', '1/8/2010', '1/11/2010',
       '1/12/2010', '1/13/2010', '1/14/2010', '1/15/2010',
       ...
       '2/4/2022', '2/7/2022', '2/8/2022', '2/9/2022', '2/10/2022',
       '2/11/2022', '2/14/2022', '2/15/2022', '2/16/2022', '2/17/2022'],
      dtype='object', name='Date', length=2809)

In [309]:
"""

MONTHLY DATA FOR FACTORS

"""

cutoff = .7

factor_df = pd.read_excel("factors_and_scores.xlsx", index_col="Date")

factor_train = factor_df[:int(len(factor_df)*cutoff)]
factor_test = factor_df[int(len(factor_df)*cutoff):]

#exposure = Rate_Exposures(train_df, 252, forward, ['Real Yield', 'Yield Curve'], "SP-500")
for_scores = Rate_Exposures(sector_df, 252, forward, ['Real Yield', 'Yield Curve'], "SP-500")

# [int(len(u.get_monthly_data(sector_df))*cutoff):]
ry_scores_test_monthly = for_scores.monthly_scores['Real Yield']
yc_scores_test_monthly = for_scores.monthly_scores['Yield Curve']
ry_scores_test_monthly = ry_scores_test_monthly.loc[ ry_scores_test_monthly.index > train_df.index[-1]]
yc_scores_test_monthly = yc_scores_test_monthly.loc[ yc_scores_test_monthly.index > train_df.index[-1]]


train_results_df = u.get_monthly_data(factor_train).copy()
train_results_df["ry_scores"] = round(train_results_df["ry_scores"], 0)
train_results_df["yc_scores"] = round(train_results_df["yc_scores"], 0)
#train_results_df

#rys = [i[0] for i in train_results_df.index]
#ycs = [i[1] for i in train_results_df.index]



In [296]:


buys = []
pred_above_market = []
exp_ret = []
top_n_sectors = 2

rates = [(i,j) for i,j in zip(train_results_df.ry_scores, train_results_df.yc_scores)]

for num, (ry, yc) in enumerate(zip(ry_scores_test_monthly, ry_scores_test_monthly)):
    
    if (ry, yc) in rates:
        
        ok = set(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == ry) & (train_results_df.yc_scores == yc)].mean().sort_values()[-top_n_sectors:].index.to_list())
        buys.append(ok)
        
        exp_ret.append(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == ry) & (train_results_df.yc_scores == yc)].mean().sort_values()[-top_n_sectors:].mean())        
    
    elif ry in rys and yc not in ycs:
        
        up_yc = yc
        down_yc = yc
        
        while (ry, up_yc) not in rates and (ry, down_yc) not in rates:
            
            up_yc += 1
            down_yc -= 1
        
        if (ry, up_yc) in rates:
            ok = set(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == ry) & (train_results_df.yc_scores == up_yc)].mean().sort_values()[-top_n_sectors:].index.to_list())
            buys.append(ok)
            exp_ret.append(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == ry) & (train_results_df.yc_scores == up_yc)].mean().sort_values()[-top_n_sectors:].mean())
            
        elif (ry, down_yc) in rates:
            ok = set(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == ry) & (train_results_df.yc_scores == down_yc)].mean().sort_values()[-top_n_sectors:].index.to_list())
            buys.append(ok)
            #exp_ret.append(train_results_df.loc[(ry,down_yc)].sort_values()[-top_n_sectors:].mean())
            exp_ret.append(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == ry) & (train_results_df.yc_scores == down_yc)].mean().sort_values()[-top_n_sectors:].mean())        
    
    elif ry not in rys and yc in ycs:

        up_ry = ry
        down_ry = ry
        
        while (up_ry, yc) not in rates and (down_ry, yc) not in rates:
            
            up_ry += 1
            down_ry -= 1
        
        if (up_ry, yc) in rates:
            ok = set(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == up_ry) & (train_results_df.yc_scores == yc)].mean().sort_values()[-top_n_sectors:].index.to_list())
            buys.append(ok)
            #exp_ret.append(train_results_df.loc[(up_ry,yc)].sort_values()[-top_n_sectors:].mean())
            exp_ret.append(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == up_ry) & (train_results_df.yc_scores == yc)].mean().sort_values()[-top_n_sectors:].mean())
            
        elif (down_ry, yc) in rates:
            
            ok = set(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == down_ry) & (train_results_df.yc_scores == yc)].mean().sort_values()[-top_n_sectors:].index.to_list())
            buys.append(ok)
            #exp_ret.append(train_results_df.loc[(down_ry,yc)].sort_values()[-top_n_sectors:].mean())
            exp_ret.append(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == down_ry) & (train_results_df.yc_scores == yc)].mean().sort_values()[-top_n_sectors:].mean())
            

    else:
        
        up_yc = yc
        down_yc = yc
        up_ry = ry
        down_ry = ry
        
        while ((up_ry, up_yc) not in rates and (down_ry, down_yc) not in rates and 
               (up_ry, down_yc) not in rates and (down_ry, up_yc) not in rates):
            
            up_yc += 1
            down_yc -= 1
            up_ry += 1
            down_ry -= 1
        
        if (up_ry, up_yc) in rates:
            ok = set(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == up_ry) & (train_results_df.yc_scores == up_yc)].mean().sort_values()[-top_n_sectors:].index.to_list())
            #ok = set(train_results_df.loc[(up_ry, up_yc)].sort_values()[-top_n_sectors:].index.to_list())
            buys.append(ok)
            #exp_ret.append(train_results_df.loc[(up_ry, up_yc)].sort_values()[-top_n_sectors:].mean())
            exp_ret.append(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == up_ry) & (train_results_df.yc_scores == up_yc)].mean().sort_values()[-top_n_sectors:].mean())
            
        elif (down_ry, down_yc) in rates:
            ok = set(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == down_ry) & (train_results_df.yc_scores == down_yc)].mean().sort_values()[-top_n_sectors:].index.to_list())
            #ok = set(train_results_df.loc[(down_ry, down_yc)].sort_values()[-top_n_sectors:].index.to_list())
            buys.append(ok)
            #exp_ret.append(train_results_df.loc[(down_ry, down_yc)].sort_values()[-top_n_sectors:].mean())
            exp_ret.append(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == down_ry) & (train_results_df.yc_scores == down_ry)].mean().sort_values()[-top_n_sectors:].mean())

        
        elif (up_ry, down_yc) in rates:
            ok = set(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == up_ry) & (train_results_df.yc_scores == down_yc)].mean().sort_values()[-top_n_sectors:].index.to_list())
            #ok = set(train_results_df.iloc[:,:-2].loc[(up_ry, down_yc)].sort_values()[-top_n_sectors:].index.to_list())
            buys.append(ok)
            #exp_ret.append(train_results_df.loc[(up_ry, down_yc)].sort_values()[-top_n_sectors:].mean())
            exp_ret.append(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == up_ry) & (train_results_df.yc_scores == down_ry)].mean().sort_values()[-top_n_sectors:].mean())

        elif (down_ry, up_yc) in rates:
            ok = set(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == down_ry) & (train_results_df.yc_scores == up_yc)].mean().sort_values()[-top_n_sectors:].index.to_list())
            #ok = set(train_results_df.loc[(down_ry, up_yc)].sort_values()[-top_n_sectors:].index.to_list())
            buys.append(ok)
            #exp_ret.append(train_results_df.loc[(down_ry, up_yc)].sort_values()[-top_n_sectors:].mean())
            exp_ret.append(train_results_df.iloc[:,:-2].loc[(train_results_df.ry_scores == down_ry) & (train_results_df.yc_scores == up_yc)].mean().sort_values()[-top_n_sectors:].mean())
 
big_df = pd.DataFrame(index = ry_scores_test_monthly.index)
big_df['exp_rets_predicted'] = exp_ret
big_df['pred_top_secs'] = buys



In [310]:

factor_test = u.get_monthly_data(factor_test.iloc[:,:-2])

factor_test = factor_test.loc[ (factor_test.index >= "2018-11") & (factor_test.index <= "2021-11")]


In [315]:


actual_returns = factor_test



In [313]:

actual_secs = []
actual_secs_above_market = []
actual_ret = []
predicted_sector_performance = []
correct_sector_performance = []

for d in actual_returns.index:
    
    #print( big_df['pred_top_secs'])

    ok = set(actual_returns.loc[str(d)].sort_values().index[-top_n_sectors:].to_list())
    
    #set(train_results_df.loc[(ry,yc)].loc[train_results_df.loc[(ry,yc)] > 0.0].index.to_list())
    #ok_cool = set(actual_returns.loc[str(d)].loc[actual_returns.loc[str(d)] > 0.0].index.to_list())
    actual_secs.append(ok)
    #actual_secs_above_market.append(ok_cool)
    
    actual_ret.append(actual_returns.loc[str(d)].sort_values()[-top_n_sectors:].mean())
    
    
    #if d in expected_results.index:
    predicted_sector_performance.append(actual_returns.loc[str(d)][list(big_df['pred_top_secs'].loc[str(d)])].mean())


actual_secs = pd.Series(actual_secs, index = actual_returns.index)
actual_ret = pd.Series(actual_ret, index = actual_returns.index)

#actual_results = pd.DataFrame()
big_df['actual_secs'] = actual_secs
big_df['actual_rets'] = actual_ret
big_df = big_df.dropna()


In [317]:

correct_sectors = []
got_one_of_top_2_sectors = []
correct_sectors_above_market = []
mean_squared_error_return = []
return_error_regular = []

for pred, act in zip(big_df['pred_top_secs'], big_df['actual_secs']):
    
    sect = pred.intersection(act)
    correct_sectors.append(len(sect))
    
    if list(pred)[-1] == list(act)[-1]:
        got_one_of_top_2_sectors.append(1)
    else:
        got_one_of_top_2_sectors.append(0)
    
for pred, act in zip(big_df['exp_rets_predicted'], big_df['actual_rets']):
    
    mean_squared_error_return.append((pred-act)**2)
    return_error_regular.append(pred-act) 


correct_sectors = pd.Series(correct_sectors, index = big_df['exp_rets_predicted'].index)
got_one_of_top_2_sectors = pd.Series(got_one_of_top_2_sectors, index = big_df['exp_rets_predicted'].index)
mean_squared_error_return = pd.Series(mean_squared_error_return, index = big_df['exp_rets_predicted'].index)
return_error_regular = pd.Series(return_error_regular, index = big_df['exp_rets_predicted'].index)

big_df['correctly_chosen_sectors'] = correct_sectors
#big_df['correctly_chosen_sectors_above_market'] = correct_sectors_above_market
big_df['predicted_sector_performance'] = predicted_sector_performance
#big_df['correct_sector_performance'] = actual_results['actual_rets']
big_df['mean_squared_error_return_prediction'] = mean_squared_error_return
big_df['return_error_regular'] = return_error_regular

big_df = big_df.dropna()



In [323]:


big_df



,exp_rets_predicted,pred_top_secs,actual_secs,actual_rets,correctly_chosen_sectors,predicted_sector_performance,mean_squared_error_return_prediction,return_error_regular
2018-11,0.114544,"{QUALITY, CASH_TO_ASSETS}","{QUALITY, CASH_TO_ASSETS}",0.117929,2,0.117929,0.000011,-0.003385
2018-12,0.087087,"{QUALITY, CASH_TO_ASSETS}","{QUALITY, CASH_TO_ASSETS}",0.063237,2,0.063237,0.000569,0.023850
2019-01,0.054267,"{QUALITY, MOMENTUM}","{QUALITY, MOMENTUM}",0.066521,2,0.066521,0.000150,-0.012254
2019-02,0.064551,"{MOMENTUM, BOOK_TO_PRICE}","{QUALITY, MOMENTUM}",0.111022,1,-0.021956,0.002160,-0.046471
2019-03,0.107600,"{DEFAULT, BOOK_TO_PRICE}","{QUALITY, MOMENTUM}",0.215252,0,-0.268332,0.011589,-0.107652
2019-04,0.107600,"{DEFAULT, BOOK_TO_PRICE}","{QUALITY, MOMENTUM}",0.238507,0,-0.312435,0.017137,-0.130907
2019-05,0.107600,"{DEFAULT, BOOK_TO_PRICE}","{QUALITY, CASH_TO_ASSETS}",0.184012,0,-0.224202,0.005839,-0.076411
2019-06,0.205168,"{QUALITY, MOMENTUM}","{SIZE, CASH_TO_ASSETS}",0.148375,0,-0.021055,0.003225,0.056793
2019-07,0.107600,"{DEFAULT, BOOK_TO_PRICE}","{SIZE, CASH_TO_ASSETS}",0.205837,0,-0.104863,0.009650,-0.098236
2019-08,0.205168,"{QUALITY, MOMENTUM}","{SIZE, CASH_TO_ASSETS}",0.239083,0,-0.027407,0.001150,-0.033916


In [322]:



big_df.predicted_sector_performance




2018-11    0.117929
2018-12    0.063237
2019-01    0.066521
2019-02   -0.021956
2019-03   -0.268332
2019-04   -0.312435
2019-05   -0.224202
2019-06   -0.021055
2019-07   -0.104863
2019-08   -0.027407
2019-09   -0.065883
2019-10   -0.048110
2019-11   -0.048569
2019-12   -0.052217
2020-01    0.143324
2020-02    0.252584
2020-03   -0.016995
2020-04   -0.423995
2020-05    0.509024
2020-06    0.409819
2020-07   -0.267421
2020-08    0.400471
2020-09    0.052345
2020-10    0.077180
2020-11    0.131182
2020-12    0.161162
2021-01    0.188832
2021-02    0.018300
Freq: M, Name: predicted_sector_performance, dtype: float64